In [2]:
import numpy as np
import pandas as pd
import nltk
import re

In [44]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Conv1D,MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
fake=pd.read_csv('Fake.csv')
real=pd.read_csv('True.csv')

In [6]:
unknown_publishers=[]
for index,row in enumerate(real.text.values):
    try:
        record=row.split(' - ',maxsplit=1)
        record[1]
        assert(len(record[0])<120)
    except:
        unknown_publishers.append(index)

In [7]:
len(unknown_publishers)

229

In [8]:
real.iloc[unknown_publishers].text

7        The following statements were posted to the ve...
8        The following statements were posted to the ve...
12       The following statements were posted to the ve...
13       The following statements were posted to the ve...
14        (In Dec. 25 story, in second paragraph, corre...
                               ...                        
20135     (Story corrects to million from billion in pa...
20500     (This Sept 8 story corrects headline, clarifi...
20667     (Story refiles to add dropped word  not , in ...
21246     (Story corrects third paragraph to show Mosul...
21339     (Story corrects to fix spelling in paragraph ...
Name: text, Length: 229, dtype: object

In [9]:
publisher= []
tmp_text= []
for index,row in enumerate(real.text.values):
    if index in unknown_publishers:
        tmp_text.append(row)
        publisher.append('Unknown')
    else:
        record=row.split(' - ',maxsplit=1)
        publisher.append(record[0].strip())
        tmp_text.append(record[1].strip())

In [10]:
real['publisher']=publisher
real['text']=tmp_text

In [11]:
empty_fake_index=[index for index, text in enumerate(fake.text.tolist()) if str(text).strip()==""]
fake.iloc[empty_fake_index]

,title,text,subject,date
10923,TAKE OUR POLL: Who Do You Think President Trum...,,politics,"May 10, 2017"
11041,Joe Scarborough BERATES Mika Brzezinski Over “...,,politics,"Apr 26, 2017"
11190,WATCH TUCKER CARLSON Scorch Sanctuary City May...,,politics,"Apr 6, 2017"
11225,MAYOR OF SANCTUARY CITY: Trump Trying To Make ...,,politics,"Apr 2, 2017"
11236,SHOCKER: Public School Turns Computer Lab Into...,,politics,"Apr 1, 2017"
...,...,...,...,...
21816,BALTIMORE BURNS: MARYLAND GOVERNOR BRINGS IN N...,,left-news,"Apr 27, 2015"
21826,FULL VIDEO: THE BLOCKBUSTER INVESTIGATION INTO...,,left-news,"Apr 25, 2015"
21827,(VIDEO) HILLARY CLINTON: RELIGIOUS BELIEFS MUS...,,left-news,"Apr 25, 2015"
21857,(VIDEO)ICE PROTECTING OBAMA: WON’T RELEASE NAM...,,left-news,"Apr 14, 2015"


In [12]:
real['text']=real['title']+" "+real['text']
fake['text']=fake['title']+" "+fake['text']

In [13]:
real['text']=real['text'].apply(lambda x: str(x).lower())
fake['text']=fake['text'].apply(lambda x: str(x).lower())

In [14]:
real['class']=1
fake['class']=0

In [15]:
real=real[['text','class']]
fake=fake[['text','class']]

In [16]:
data=real.append(fake,ignore_index=True)#merging data
len(data)

C:\Users\faiza\AppData\Local\Temp\ipykernel_47360\1826422205.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=real.append(fake,ignore_index=True)#merging data


44898

In [17]:
label=data['class']
text=data['text']
len(label)

44898

In [18]:
import preprocess_kgptalkie as ps

In [19]:
data['text'].apply(lambda x: ps.remove_special_chars(x))

0        as us budget fight looms republicans flip thei...
1        us military to accept transgender recruits on ...
2        senior us republican senator let mr mueller do...
3        fbi russia probe helped by australian diplomat...
4        trump wants postal service to charge much more...
                               ...                        
44893    mcpain john mccain furious that iran treated u...
44894    justice yahoo settles email privacy classactio...
44895    sunnistan us and allied safe zone plan to take...
44896    how to blow 700 million al jazeera america fin...
44897    10 us navy sailors held by iranian military si...
Name: text, Length: 44898, dtype: object

In [54]:
import gensim
y=data['class']
x=data['text']

In [64]:
y

0        1
1        1
2        1
3        1
4        1
        ..
44893    0
44894    0
44895    0
44896    0
44897    0
Name: class, Length: 44898, dtype: int64

In [55]:
x=x.apply(lambda it: str(it).lower())

In [57]:
X_train,X_test,y_train,y_test=train_test_split(x,y)

In [59]:
###count vectorization(Bag of WORDS)
vec=CountVectorizer(lowercase=False)
vec.fit(list(X_train)+list(X_test))
x_train_vec=vec.transform(X_train)
x_test_vec=vec.transform(X_test)

In [60]:
##model-1####
from sklearn import svm
svm=svm.SVC(kernel='rbf',probability=True)
prob=svm.fit(x_train_vec,y_train).predict_proba(x_test_vec)
y_pred_svm=svm.predict(x_test_vec)

In [61]:
accuracy_score(y_test,y_pred_svm)

0.9823608017817372

In [66]:
vectorizer=TfidfVectorizer(min_df=0.01,lowercase=False,max_features=1000)
vectorizer.fit(list(X_train)+list(X_test))
x_train_tfidf=vectorizer.transform(X_train)
x_test_tfidf=vectorizer.transform(X_test)

In [67]:
###mode-2####
from sklearn import svm
svm=svm.SVC(kernel='rbf',probability=True)
prob=svm.fit(x_train_tfidf,y_train).predict_proba(x_test_tfidf)
y_pred_svm=svm.predict(x_test_tfidf)

In [68]:
accuracy_score(y_test,y_pred_svm)

0.9878841870824053